# Predicted Word Associates of Texts, based on Word Cooccurrence Statistics

Here, we calculate the predicted word associates of the texts that are used in our text memory experiment. These predictions can be used to test a word association account of text memory, where word associations are defined in terms of cooccurrence statistics in the language.

## A probabilistic model of word cooccurrences 

$$
\newcommand{\data}{\mathcal{D}}
\newcommand{\Prob}[1]{\mathrm{P}( #1 )}
\newcommand{\given}{\vert}
$$

If $\mathcal{D}$ is a corpus of $J$ texts, i.e., $\mathcal{D} = \textrm{text}_1, \textrm{text}_2 \ldots \textrm{text}_j$, the probability that word $w_k$ and $w_l$ both appear in text $j$ is 
$$
\Prob{w_k, w_l \given \textrm{text}_j} = \int \Prob{w_k, w_l \given \pi_j} \Prob{\pi_j \given \data} d\pi_j
$$
and their average probability of co-occurrence in the corpus is 
$$
\Prob{w_k, w_l} = \frac{1}{J} \sum_{j=1}^J \Prob{w_k, w_l \given \textrm{text}_j}.
$$

Here, $\Prob{\pi_j \given \data}$ is the posterior probability of $\pi_j$, which is a categorical distribution over the length $V$ vocabulary. In other words, each text is modelled as a probability distribution over the entire vocabulary of words, and we infer this distribution from the corpus as follows:
$$
\begin{align}
\mathrm{P}(\pi_j \vert \mathcal{D}) 
&= \int \mathrm{P}(\pi_j \vert \mathcal{D}, a, m)\mathrm{P}(a, m \vert \mathcal{D}),\\
&\approx \mathrm{P}(\pi_j \vert \mathcal{D}, \hat{a}, \hat{m})
\end{align}
$$
where $\hat{a}$ and $\hat{m}$ are the posterior means of $a$ and $m$ (conditioned on $\mathcal{D}$), respectively, and
$$
\mathrm{P}(\pi_j \vert text_j) \propto \prod_{i=1}^{n_j} \mathrm{P}(w_{ji} \vert \pi_j) \mathrm{P}(\pi_j \vert a, m).
$$
If $\mathrm{P}(\pi_j \vert a, m)$ is a Dirichlet distribution with location parameter $m$ and concentration parameter $a$, then the posterior distribution is 
$$
\mathrm{P}(\pi_j \vert text_j) = \textrm{Dirichlet}(R_{j1} + a m_{1}, R_{j2} + a m_{2} \ldots R_{jV} + a m_{V})
$$
From this, we have
$$
\begin{align}
\Prob{w_k, w_l \given \textrm{text}_j} &= \int \Prob{w_k, w_l \given \pi_j} \Prob{\pi_j \given \data, \hat{a}, \hat{m}} d\pi_j,\\
&=\frac{\Gamma(R_{j\cdot} + a)}{\prod_{v=1}^V \Gamma(R_{jv} + am_v)}
\times
\frac{\prod_{v=1}^V \Gamma(R_{jv} + a m_{v} + \mathbb{I}(w_k = v) + \mathbb{I}(w_l = v) )}{\Gamma(R_{j\cdot} + a + 2) },\\
&=\frac{\Gamma(R_{j\cdot} + a)}{\Gamma(R_{j\cdot} + a + 2) }
\times \prod_{v=1}^V \frac{\Gamma(R_{jv} + a m_{v}+ \mathbb{I}(w_k = v) + \mathbb{I}(w_l = v)) }{\Gamma(R_{jv} + am_v)}
,\\
&=\frac{(R_{jk} + a m_{k}) (R_{jl} + a m_{l}+ \mathbb{I}(k = l))}{(R_{j\cdot} + a + 1) (R_{j\cdot} + a))}
\end{align}
$$
and so
$$
\Prob{w_k, w_l} = \frac{1}{J} \sum_{j=1}^J \frac{(R_{jk} + a m_{k}) (R_{jl} + a m_{l}+ \mathbb{I}(k = l))}{(R_{j\cdot} + a + 1) (R_{j\cdot} + a))}
$$

## Preliminaries


In [1]:
from __future__ import division

# Standard library imports
import os
import cPickle as pickle
import itertools
from collections import defaultdict

# Third party imports
import configobj
import numpy
from scipy.special import gamma, gammaln
from scipy.sparse import coo_matrix
from gustav import models, utils # Available at https://lawsofthought.github.io/gustavproject/

# Local imports
from utils import utils
from utils import datautils
from utils.datautils import tokenize

We can use the following to verify the algebraic derivations above.

In [2]:
def check_the_math(iterations=100):

    '''
    Test the algebraic derivations shown above. Added for those in doubt.
    '''
    
    def long_way(Rjdot, a, Rj, m, k, l, V):

        wk = numpy.zeros(V)
        wl = numpy.zeros(V)
        wk[k] = 1
        wl[l] = 1

        return numpy.exp(gammaln(Rjdot + a) - gammaln(Rjdot + a + 2) +\
            sum([gammaln(Rj[v] + a*m[v] + wk[v] + wl[v]) - gammaln(Rj[v] + a*m[v]) for v in xrange(V)]))

    def quick_way(Rjdot, a, Rj, m, k, l, V):

        I = 1*(k == l)

        return ((Rj[k] + a*m[k]) * (Rj[l] + a*m[l] + I)) / ((Rjdot + a + 1) * (Rjdot + a))

    for iteration in xrange(iterations):

        Rjdot = numpy.random.randint(0, 20)
        a = numpy.random.uniform(0.1, 10.0)
        V = numpy.random.randint(1000, 10000)
        Rj = numpy.random.randint(0, 10, size=V)
        m = numpy.random.rand(V)

        k = numpy.random.randint(0, V)
        l = numpy.random.randint(0, V)

        assert numpy.isclose(long_way(Rjdot, a, Rj, m, k, l, V),
                             quick_way(Rjdot, a, Rj, m, k, l, V))
    
        assert numpy.isclose(long_way(Rjdot, a, Rj, m, k, k, V),
                             quick_way(Rjdot, a, Rj, m, k, k, V))

check_the_math() # 100 iterations should take around 10 secs

Get the necessary data files, and process them if necessary.

In [3]:
url_root = 'http://www.lawsofthought.org/shared'

cache_directory = '../_cache'

filenames = {
    'experiment_cfg' : [('Brismo.cfg',
                         '909d9f8de483c4547f26fb4c34b91e12908ab5c144e065dc0fe6c1504b1f22c9')],
    'text-corpus' : [('bnc_texts_78723408_250_500.txt.bz2', 
                      'dd8806f51088f7c8ad6c1c9bfadb6680c44bc5fd411e52970ea9c63596c83d34')],
    'vocabulary' : [('bnc_vocab_49328.txt',
                     '55737507ea9a2c18d26b81c0a446c074c6b8c72dedfa782c763161593e6e3b97')]
}

utils.curl(url_root, 
           filenames['experiment_cfg'] + filenames['text-corpus'] + filenames['vocabulary'], 
           cache=cache_directory,
           verbose=False)

memoranda = configobj.ConfigObj(os.path.join(cache_directory, 
                                             filenames['experiment_cfg'][0][0]))['text_memoranda']

text_filename = os.path.join(cache_directory, 
                             os.path.splitext(filenames['text-corpus'][0][0])[0])

vocabulary_filename = os.path.join(cache_directory, 
                                   filenames['vocabulary'][0][0])

vocabulary = open(vocabulary_filename).read().split()
vocab = datautils.Vocab(vocabulary)

text_to_words = lambda text: [word for word in tokenize(text) if word in vocab.word2index]

texts_as_words = {}
for text_name in memoranda:
    texts_as_words[text_name] = text_to_words(memoranda[text_name]['text'])

prime_words = sorted(set(itertools.chain(*texts_as_words.values())))

Make a corpus for use in a Multinomial Dirichlet Compound model, unless it is already made.

In [4]:
sparse_count_matrix_filename = 'bnc_78723408_250_500_vocab49328_sparse_matrix_count'
sparse_count_matrix_filename_path = os.path.join(cache_directory, sparse_count_matrix_filename + '.npz')

if not os.path.exists(sparse_count_matrix_filename_path):

    utils.SparseCountMatrix.new(text_filename = text_filename,
                                vocabulary_filename = vocabulary_filename,
                                save_filename = sparse_count_matrix_filename)

## Dirichlet multinomial compound (DMC) language model

In [5]:
# Some helper functions

def multinomial_dirichlet_compound_burnin(args):

    cache_directory = '_cache' # From the POV of ipcluster
    
    iterations, seed = args
    
    model = models.MultinomialDirichletCompoundModel()

    model.load_data(os.path.join(cache_directory, sparse_count_matrix_filename + '.npz'))

    model.initialize(seed)

    model.update(iterations=iterations, sample_c=True)
    
    return model

def multinomial_dirichlet_compound_sample(args):
    
    model, iterations, thin = args 
    
    samples = model.sample(number_of_samples=iterations, thin=thin)

    return samples

def make_burnin_args(iterations, seed, nchains=3):
    
    random = numpy.random.RandomState(seed=seed)
    seeds = random.randint(101, 100001, size=nchains)
    
    return zip([iterations]*nchains, seeds)
    
def make_sampler_args(models, iterations, thin, nchains=3):
    return zip(models, [iterations]*nchains, [thin]*nchains)

def gelman_diag(psi):
    
    '''The Gelman-Rubin convergence diagnostic.'''
    
    m, n = psi.shape
    
    B = n * numpy.var(psi.mean(1), ddof=1)
    
    W = psi.var(1, ddof=1).mean() 

    V = (n-1)/n * W + B/n    

    return numpy.sqrt(V/W)

### MCMC sampler

Run 3 parallel chains. Burn in each chain for 100 iterations, sample *b*, *c* and *psi* from each chain 100 times, dropping every second sample. This may seem like a small number of samples. However, from previous experience, we know that convergence will be rapid and the posterior distribution of the variables is sharply peaked.

#### Some parameters

* The **master seed**, which will make seeds that make seeds that make seeds, and son on ensuring complete reproducibility
* The number of chains

In [6]:
MASTER_SEED = 27431
NCHAINS = 3
NCORES = 16
N_BURN_IN = 100
N_SAMPLE = 100
SAMPLE_THIN = 2

samples_save_filename_basename = 'multinomial_dirichlet_chains_seed_%d_burn_%d_sample_%d_thin_%d.pkl' % (MASTER_SEED,
                                                                                                         N_BURN_IN,
                                                                                                         N_SAMPLE,
                                                                                                         SAMPLE_THIN)

samples_save_filename = os.path.join(cache_directory, 
                                     samples_save_filename_basename)

In [7]:
if not os.path.exists(samples_save_filename):

    # Start a cluster on the command line with "ipcluster start -n NCORES" 
    # where NCORES is at least as large as NCHAINS
    
    # Each chain takes about 6 hours (give or take)

    from ipyparallel import Client

    clients = Client()

    clients.block = True

    clients[:].push(dict(sparse_count_matrix_filename = sparse_count_matrix_filename))

    with clients[:].sync_imports():
        from gustav import models

    view = clients.load_balanced_view()

    models = view.map(multinomial_dirichlet_compound_burnin, 
                      make_burnin_args(N_BURN_IN, seed=MASTER_SEED, nchains=NCHAINS))

    samples = view.map(multinomial_dirichlet_compound_sample,
                       make_sampler_args(models, N_SAMPLE, SAMPLE_THIN))

    with open(samples_save_filename, 'wb') as f:
        pickle.dump(samples, file=f, protocol=2)
        
else:
    
    with open(samples_save_filename, 'rb') as f:
        samples = pickle.load(f)

### Convergence diagnostics

We'll check if *b*, *c*, and each variable of the vector variable *psi* have converged using the Gelman Rubin diagnostic. For the *psi*, we will also check if the means of samples of *psi* across the three chains are more or less identical. 

In [8]:
def get_gelman_diag(key='b'):
    return gelman_diag(numpy.array([samples[k][key] for k in xrange(NCHAINS)]))

In [9]:
get_gelman_diag('b')

1.0141675325123143

In [10]:
get_gelman_diag('c')

0.99785578883547243

In [11]:
numpy.max([gelman_diag(numpy.array([[psi[i] for psi in samples[k]['psi']] for k in xrange(NCHAINS)]))
 for i in xrange(len(samples[0]['psi'][0]))])

1.049718484858517

In [12]:
numpy.corrcoef(numpy.array([numpy.array(samples[k]['psi']).mean(0) for k in xrange(NCHAINS)]))

array([[ 1.        ,  0.99999807,  0.99999808],
       [ 0.99999807,  1.        ,  0.99999809],
       [ 0.99999808,  0.99999809,  1.        ]])

### Calculate sample mean of *b* and *psi*

In [13]:
b = numpy.array([samples[k]['b'] for k in (0, 1, 2)]).mean()
psi = numpy.array([samples[k]['psi'] for k in (0, 1, 2)]).mean(axis=(0,1))

## Make predictions

In [14]:
model = models.MultinomialDirichletCompoundModel()
model.load_data(os.path.join(cache_directory, sparse_count_matrix_filename + '.npz'))

R = coo_matrix((model.values, (model.rows, model.cols)), shape=(model.J, model.V)).tocsr()

$$
\begin{align}
\Prob{w_l \given w_k} &= \frac{\Prob{w_k, w_l}}{\Prob{w_k}} = \frac{\Prob{w_k, w_l}}{\sum_{\{w_l\}} \Prob{w_k, w_l}},\\
\Prob{w_k, w_l} &= \frac{1}{J} \sum_{j=1}^J \frac{(R_{jk} + a m_{k}) (R_{jl} + a m_{l}+ \mathbb{I}(k = l))}{(R_{j\cdot} + a + 1) (R_{j\cdot} + a))}
\end{align}
$$

In [15]:
def Pr_wl_given_wk(R, word_indices, b, psi):
    
    '''
    Return predicted probability of all words conditioned on word k.
    In other words, what is the predicted probability of observing any
    given word given that we have observed word k.
    
    '''

    J, V = R.shape
    
    k_len = len(word_indices)
    
    I = numpy.zeros((k_len, V))
        
    for k in xrange(k_len):
        I[k, word_indices[k]] = 1

    p = numpy.zeros((k_len, V))
    
    for j in xrange(J):
    
        Rjam = (R[j] + b*psi).A.flatten()
        Rjdot = R[j].sum()

        Z = (Rjdot + b + 1) * (Rjdot + b)

        for k in xrange(k_len):
            p[k] += Rjam[word_indices[k]] * (Rjam + I[k]) / Z
        
    for k in xrange(k_len):
        p[k] = p[k]/J
        p[k] = p[k]/p[k].sum()
    
    return p

In [16]:
#p = Pr_wl_given_wk(R, (10, 101, 10001), b, psi)

In [17]:
_prime_words = prime_words[:] # copy

assignments = defaultdict(list)
for k in itertools.cycle(xrange(NCORES-2)):
    assignments[k].append(vocab.word2index[_prime_words.pop()])
    
    if not _prime_words:
        break

In [301]:
args = []

for assignment in assignments.values():    
    args.append((R, assignment, b, psi))

def foo(args):
    R, word_indices, b, psi = args
    return word_indices, Pr_wl_given_wk(R, word_indices, b, psi)

In [303]:
# This will take around 2 to 3 hrs on a 16 core system
from ipyparallel import Client

clients = Client()

clients.block = True

clients[:].push(dict(Pr_wl_given_wk = Pr_wl_given_wk))

with clients[:].sync_imports():
    import numpy

view = clients.load_balanced_view()

results = view.map(foo, args)

importing numpy on engine(s)


In [304]:
predictions = {}
for result in results:
    for i,v in enumerate(result[0]):
        predictions[vocab.index2word[v]] = result[1][i]

In [59]:
assert len(predictions.keys()) == len(prime_words)

In [67]:
def get_topic(p, k=10):
    return [vocab.index2word[w] for w in numpy.flipud(p.argsort())[:k]]

In [135]:
print(get_topic(predictions['war'], k=100))

['war', 'time', 'people', 'world', 'government', 'life', 'day', 'found', 'left', 'home', 'set', 'house', 'local', 'system', 'national', 'public', 'party', 'head', 'country', 'social', 'called', 'children', 'told', 'form', 'hand', 'political', 'looked', 'days', 'power', 'family', 'support', 'major', 'women', 'held', 'change', 'times', 'company', 'night', 'money', 'control', 'information', 'business', 'means', 'school', 'eyes', 'cent', 'john', 'development', 'view', 'service', 'including', 'period', 'main', 'terms', 'mind', 'half', 'past', 'question', 'round', 'policy', 'level', 'result', 'economic', 'act', 'months', 'effect', 'real', 'position', 'week', 'feel', 'brought', 'office', 'sense', 'difficult', 'water', 'hard', 'future', 'labour', 'market', 'taking', 'words', 'provide', 'close', 'body', 'century', 'society', 'book', 'matter', 'idea', 'late', 'international', 'law', 'father', 'line', 'front', 'south', 'experience', 'white', 'process', 'west']
